**Cài đặt thư viện UnderTheSea để hỗ trợ tách từ trong tiếng Việt. Xem thêm các tính năng của thư viện tại: https://github.com/undertheseanlp/underthesea**

In [ ]:
!pip install underthesea

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.0 MB 4.9 MB/s 
     |████████████████████████████████| 235 kB 49.4 MB/s 
     |████████████████████████████████| 1.0 MB 36.1 MB/s 
     |████████████████████████████████| 591 kB 41.9 MB/s 


**Import các thư viện cần thiết**

In [ ]:
import os
import re
import math
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from underthesea import word_tokenize

**Cho một tập dữ liệu (D) mẫu như bến dưới**

In [ ]:
# Khai báo tập tài liệu/văn bản
D = [
    'Khai phá dữ liệu (data mining) Là quá trình tính toán để tìm ra các mẫu trong các bộ dữ liệu lớn liên quan đến các phương pháp tại giao điểm của máy học, thống kê và các hệ thống cơ sở dữ liệu. Đây là một lĩnh vực liên ngành của khoa học máy tính.',
    'Truy hồi thông tin (information retrieval) là hoạt động thu thập các nguồn thông tin liên quan đến một thông tin cần tìm kiếm, có thể dựa trên dữ liệu và trên việc đánh chỉ mục toàn văn.',
    'Xử lý ngôn ngữ tự nhiên (natural language processing) là một nhánh cực kỳ quan trọng của trí tuệ nhân tạo (AI), là giao điểm của ngôn ngữ học và khoa học máy tính. NLP làm nhiệm vụ xử lý và phân tích một lượng lớn dữ liệu ngôn ngữ tự nhiên để bắt chước các tương tác giữa con người theo cách giống con người.'
]

# Xác định số lượng tài liệu/văn bản
doc_size = len(D)

**Tiến hành tách từ và xây dựng tập từ vựng ở dạng chỉ mục ngược (inverted_index)**

In [ ]:
# Xây dựng tập từ vựng (V) dạng cấu trúc chỉ mục ngược
# (V) là cấu trúc dữ liệu dạng dictionary <key: token_1, value: [(doc_idx_1, tf(token_1)), (doc_idx_2, tf(token_1)), v.v.>
inverted_index = {}

# Chúng cũng cần xác định trọng số lớn nhất của từ xuất hiện trong mỗi tài liệu/văn bản để cập nhật lại tf
# Cấu trúc dữ liệu dạng dictionary <key: doc_idx, value: <key: token, value: token_freq>>
doc_idx_token_token_freq = {}

# Tiến hành duyệt qua từng văn bản để xây dựng tập từ vựng
for doc_idx, doc in enumerate(D):
  # Để thuận tiện ta sẽ chuyển tất cả các từ trong các tài liệu/văn bản về dạng lowercase
  doc = doc.lower()
  
  # Xóa bỏ các ký tự đặc biệt
  clean_doc = re.sub('\W+',' ', doc)
  
  # Tiến hành dùng thư viện UnderTheSea để tách các từ/token trong tiếng Việt
  tokens = word_tokenize(clean_doc)
  
  # Tiến hành thay thế các khoảng trắng ' ' trong các từ ghép thành '_'
  tokens = [token.replace(' ', '_') for token in tokens]

  # Duyệt qua từng token
  for token in tokens:
    # Kiểm tra xem token đã tồn tại trong tập từ vựng (V) hay chưa
    if token not in inverted_index.keys():
      inverted_index[token] = [(doc_idx, 1)]
    else:
      # Kiểm tra xem tài liệu doc_idx đã có trong danh sách các tài liệu chỉ mục ngược của token này hay chưa
      is_existed = False
      for inverted_data_idx, (target_doc_idx, target_tf) in enumerate(inverted_index[token]):
        if target_doc_idx == doc_idx:
          # Tăng tần số xuất hiện của token trong tài liệu (target_doc_idx) lên 1
          target_tf+=1
          # Cập nhật lại dữ liệu
          inverted_index[token][inverted_data_idx] = (target_doc_idx, target_tf)
          is_existed = True
          break
      # Trường hợp chưa tồn tại
      if is_existed == False:
        inverted_index[token].append((doc_idx, 1))
    
      if doc_idx not in doc_idx_token_token_freq.keys():
        doc_idx_token_token_freq[doc_idx] = {}
        doc_idx_token_token_freq[doc_idx][token] = 1
      else:
        if token not in doc_idx_token_token_freq[doc_idx].keys():
          doc_idx_token_token_freq[doc_idx][token] = 1
        else:
          doc_idx_token_token_freq[doc_idx][token] += 1

**Cập nhật lại tf của từng token trong danh sách các tài liệu/văn bản chỉ mục ngược. Vì hiện tại giá trị tf của chúng ta chỉ là số lần xuất hiện của từ/token trong mỗi tài liệu**

**Nhưng tf cửa 1 từ (i) trong tài liệu/văn bản (j), ký hiệu: ($tf_{ij}$) lại được xác định là**: $$tf_{ij}=\frac{f_{ij}}{max_i(f_{ij})}$$

In [ ]:
# Hàm tìm từ khóa/token xuất hiện nhiều nhất trong một tài liệu/văn bản (doc_idx)
def find_max_freq_token(doc_idx):
  max_freq_token = ''
  max_freq = 0
  tokens = doc_idx_token_token_freq[doc_idx]
  for token in doc_idx_token_token_freq[doc_idx].keys():
    if doc_idx_token_token_freq[doc_idx][token] > max_freq:
      max_freq_token = token
      max_freq = doc_idx_token_token_freq[doc_idx][token]
  return (max_freq_token, max_freq)

# Cấu trúc dữ liệu dạng dictionary <key: doc_idx, value: (max_freq_token, max_freq)>
doc_idx_max_freq_token = {}
for doc_idx, doc in enumerate(D):
  doc_idx_max_freq_token[doc_idx] = find_max_freq_token(doc_idx)

# Cập nhật lại tf của từng token trong danh sách các tài liệu/văn bản chỉ mục ngược
for token in inverted_index.keys():
  D_t = inverted_index[token]
  for inverted_data_idx, (doc_idx, tf) in enumerate(D_t):
    # Cập nhật lại trọng số tf của token đang xét
    (max_freq_token, max_freq) = doc_idx_max_freq_token[doc_idx]
    update_tf = tf / max_freq
    # Cập nhật lại dữ liệu
    inverted_index[token][inverted_data_idx] = (doc_idx, update_tf)

**In danh sách các token/từ khóa và các tài liệu/văn bản mà nó xuất hiện dạng chỉ mục ngược ra màn hình**

In [ ]:
for token in inverted_index.keys():
  print(token, '->', inverted_index[token])

khai_phá -> [(0, 0.3333333333333333)]
dữ_liệu -> [(0, 1.0), (1, 0.5), (2, 0.5)]
data -> [(0, 0.3333333333333333)]
mining -> [(0, 0.3333333333333333)]
là -> [(0, 0.6666666666666666), (1, 0.5), (2, 1.0)]
quá_trình -> [(0, 0.3333333333333333)]
tính_toán -> [(0, 0.3333333333333333)]
để -> [(0, 0.3333333333333333), (2, 0.5)]
tìm -> [(0, 0.3333333333333333)]
ra -> [(0, 0.3333333333333333)]
các -> [(0, 1.3333333333333333), (1, 0.5), (2, 0.5)]
mẫu -> [(0, 0.3333333333333333)]
trong -> [(0, 0.3333333333333333)]
bộ -> [(0, 0.3333333333333333)]
lớn -> [(0, 0.3333333333333333), (2, 0.5)]
liên_quan -> [(0, 0.3333333333333333), (1, 0.5)]
đến -> [(0, 0.3333333333333333), (1, 0.5)]
phương_pháp -> [(0, 0.3333333333333333)]
tại -> [(0, 0.3333333333333333)]
giao_điểm -> [(0, 0.3333333333333333), (2, 0.5)]
của -> [(0, 0.6666666666666666), (2, 1.0)]
máy -> [(0, 0.3333333333333333)]
học -> [(0, 0.3333333333333333)]
thống_kê -> [(0, 0.3333333333333333)]
và -> [(0, 0.3333333333333333), (1, 0.5), (2, 1.0)]
hệ_

**Áp dụng tập từ vựng ở dạng dữ liệu chỉ mục ngược và tính trọng số TF-IDF của mỗi từ/token với mỗi văn bản. Với công thức TF-IDF được xác định như sau**
$$ TF-IDF = TF \times IDF $$
**Và**: $$IDF = log_{2}(\frac{|D|}{df_{i}})$$

In [ ]:
for token in inverted_index.keys():
  # Lấy ra danh sách các tài liệu/văn bản mà từ vựng này xuất hiện
  D_t = inverted_index[token]
  # Từ đó, ta xác định được doc_freq (df)
  doc_freq = len(D_t)
  for (doc_idx, tf) in D_t:
    idf = math.log((doc_size / doc_freq),2)
    tfidf = tf * idf
    print('Từ: [',token, ']-> tài liệu số: [', doc_idx, '], TF-IDF = [', tfidf, ']')
  print('---')

Từ: [ khai_phá ]-> tài liệu số: [ 0 ], TF-IDF = [ 0.5283208335737187 ]
---
Từ: [ dữ_liệu ]-> tài liệu số: [ 0 ], TF-IDF = [ 0.0 ]
Từ: [ dữ_liệu ]-> tài liệu số: [ 1 ], TF-IDF = [ 0.0 ]
Từ: [ dữ_liệu ]-> tài liệu số: [ 2 ], TF-IDF = [ 0.0 ]
---
Từ: [ data ]-> tài liệu số: [ 0 ], TF-IDF = [ 0.5283208335737187 ]
---
Từ: [ mining ]-> tài liệu số: [ 0 ], TF-IDF = [ 0.5283208335737187 ]
---
Từ: [ là ]-> tài liệu số: [ 0 ], TF-IDF = [ 0.0 ]
Từ: [ là ]-> tài liệu số: [ 1 ], TF-IDF = [ 0.0 ]
Từ: [ là ]-> tài liệu số: [ 2 ], TF-IDF = [ 0.0 ]
---
Từ: [ quá_trình ]-> tài liệu số: [ 0 ], TF-IDF = [ 0.5283208335737187 ]
---
Từ: [ tính_toán ]-> tài liệu số: [ 0 ], TF-IDF = [ 0.5283208335737187 ]
---
Từ: [ để ]-> tài liệu số: [ 0 ], TF-IDF = [ 0.1949875002403854 ]
Từ: [ để ]-> tài liệu số: [ 2 ], TF-IDF = [ 0.2924812503605781 ]
---
Từ: [ tìm ]-> tài liệu số: [ 0 ], TF-IDF = [ 0.5283208335737187 ]
---
Từ: [ ra ]-> tài liệu số: [ 0 ], TF-IDF = [ 0.5283208335737187 ]
---
Từ: [ các ]-> tài liệu số: [ 0 ],